In [ ]:
from typing import List

import torch
from transformers import AutoModelForCausalLM
from src.data_tokenizer import RapDataTokenizer
from src.lyrics_cleaner import clean_generated_lyrics
# используем тот же GPU, что и для обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# путь, куда была сохранена модель после обучения
MODEL_PATH = "./final_model"
MODEL_NAME = "sberbank-ai/rugpt3small_based_on_gpt2"

# загружаем токенизатор и модель
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH).to(device)
rap_tokenizer = RapDataTokenizer(MODEL_NAME)

print(f" Модель и токенизатор готовы на устройстве: {device}")

In [ ]:
def generate_and_clean_rap_lyrics(
    prompt: str,
    max_tokens: int = 100,
    temperature: float = 0.85,
    top_k: int = 40,
    top_p: float = 0.85,
    repetition_penalty: float = 1.8
) -> List[str]:
    """
    Создает лирику на основе промпта, генерирует текст и очищает его от токенов.
    Использует глобальные объекты 'model', 'rap_tokenizer', и 'device'.
    """

    # 1. Подготовка ввода
    inputs = rap_tokenizer.tokenizer(
        prompt,
        return_tensors='pt',
        padding=True,
        truncation=True
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Максимальная длина генерации (длина промпта + желаемое число новых токенов)
    max_length = input_ids.shape[1] + max_tokens

    # 2. Генерация
    model.eval()
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            num_return_sequences=1,
            pad_token_id=rap_tokenizer.tokenizer.pad_token_id
        )

    # 3. Декодирование сырого вывода
    raw_output = rap_tokenizer.tokenizer.decode(output_ids[0], skip_special_tokens=False)

    # 4. Очистка и форматирование
    cleaned_lyrics = clean_generated_lyrics(raw_output)

    return cleaned_lyrics

In [ ]:
test_depo = (
    f"<META> [Boulevard Depo] [Dark, Lyrical, Philosophical, Abstract, Conceptual, Cloud Rap] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Мой флоу течет сквозь стены, будто плазма,\n"
    f"Нам осталась только пыль из мира кармы.\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>" # Просим модель продолжить припевом
)

print("--- ГЕНЕРАЦИЯ DEPO ---")
lyrics_kai = generate_and_clean_rap_lyrics(prompt=test_depo, repetition_penalty=1.9, temperature=1.1)
print('\n'.join(lyrics_kai))

In [ ]:
prompt_kai_abstract = (
    f"<META> [Kai Angel] [Industrial, Dark, Conceptual] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Зачем смотреть вперед, если там только ноль?\n"
    f"Я заблокировал вход в свой персональный мозг, это контроль.\n"
    f"Моя машина — это танк, и он едет мимо всех.\n"
)


print("--- ГЕНЕРАЦИЯ KAI ANGEL ABSTRACT ---")
lyrics_kai = generate_and_clean_rap_lyrics(prompt=prompt_kai_abstract, repetition_penalty=1.9, temperature=1.4)
print('\n'.join(lyrics_kai))

In [ ]:
prompt_kai_chorus = (
    f"<META> [Kai Angel] [Industrial, Cloud Rap, New School] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Красный код на студии, красный свет в моих глазах.\n"
    f"Я не сплю, я строю мир на твоих костях.\n"
    f"Они думали, что знают, но не знали ничего.\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)
print("--- ГЕНЕРАЦИЯ KAI ANGEL CHORUS ---")
lyrics_kai = generate_and_clean_rap_lyrics(prompt=prompt_kai_chorus, repetition_penalty=2.1, temperature=1.9)
print('\n'.join(lyrics_kai))

In [ ]:
prompt_pharaoh = (
    f"<META> [Pharaoh] [Emo Trap, Dark, Melancholic, Cloud Rap] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Мне так одиноко, я сжёг все мосты, о да\n"
    f"Ты даже не знаешь, сколько лет мне теперь\n"
    f"Я мог бы тебя проводить в темноту, но не в дверь\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)
print("\n--- PHARAOH (ДИКО, НАПРИМЕР) ---")
lyrics_pharaoh = generate_and_clean_rap_lyrics(
    prompt=prompt_pharaoh,
    temperature=1.0,
    repetition_penalty=1.5
)
print('\n'.join(lyrics_pharaoh))

In [ ]:
prompt_saluki = (
    f"<META> [Saluki] [Trap, New School, Smooth, Lyrical] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Я выкурил все огни в этих окнах и вышел\n"
    f"Теперь мой бренд — это знак, он дороже всех слов\n"
    f"Мне нужна только ты, но тебя слишком много вокруг\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)
print("\n--- SALUKI (ВЕНЕРА) ---")
lyrics_saluki = generate_and_clean_rap_lyrics(
    prompt=prompt_saluki,
    temperature=0.7,
    repetition_penalty=1.3
)
print('\n'.join(lyrics_saluki))

In [ ]:
prompt_husky = (
    f"<META> [Husky] [Experimental, Aggressive, Social, Dark] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Нам остался только бетон и холодный свет из окна\n"
    f"Здесь всё одно: и смерть, и жизнь, и моя голова\n"
    f"Каждый день, как копия, каждый шаг, как ловушка\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)
print("\nХАСКИ (ПАНЕЛЬКА) ---")
lyrics_husky = generate_and_clean_rap_lyrics(
    prompt=prompt_husky,
    temperature=0.45,
    repetition_penalty=1.8
)
print('\n'.join(lyrics_husky))

In [ ]:
prompt_dark_prince_true = (
    f"<META> [Тёмный Принц] [Trap, Aggressive, Scammer, New School] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Моя цель — её разрушить.\n"
    f"Факаю тебя в маске, что с улыбкою по уши.\n"
    f"Я воркаю один, за минуту три лога, да, мои пальцы в крови.\n"
    f"Мне нужен профит, нужен обход Vinted.\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)

# ----------------------------------------------------
# Запуск теста с новым, точным контекстом
# ----------------------------------------------------
print("--- ТЕСТ: ТЁМНЫЙ ПРИНЦ (SCAMMER TRAP) ---")
lyrics_pharaoh_papa_true = generate_and_clean_rap_lyrics(
    prompt=prompt_dark_prince_true,
    temperature=0.9,
    repetition_penalty=1.8
)

print('\n'.join(lyrics_pharaoh_papa_true))